## Value Investing 

In [175]:
import pandas as pd
import numpy as np
import yfinance as yf
import math


## Loading the Stocks list

In [176]:
tickers = pd.read_csv("top_50_indian_stocks.csv")
tickers.head(5)

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank


In [177]:

# def fetch_values_of_stocks(tickers):
#     value_cols = [
#         "Ticker",
#         "Price",
#         "PE-Ratio",
#         "PB-Ratio",
#         "PS-Ratio",
#         "EV/EBITDA",
#         "EV/GP"
#     ]
#     value_df = pd.DataFrame(columns=value_cols)
#     for ticker in tickers:
#         try:
#             stock = yf.Ticker(ticker)
#             price = stock.history(period="1d")['Close'].iloc[-1] if not stock.history(period="1d").empty else np.nan
#             # ... (rest of your calculations for pe_ratio, pb_ratio, etc.)

#             new_row = pd.Series(  # Create a Series with the *values*
#                 data=[
#                     ticker,  # The ticker value
#                     price,
#                     pe_ratio,
#                     pb_ratio,
#                     ps_ratio,
#                     evEbitda,
#                     evGrossProfit
#                 ],
#                 index=value_cols  # Use value_cols as the index
#             )

#             value_df = pd.concat([value_df, new_row.to_frame().T], ignore_index=True) # More efficient way to append a row
#         except (IndexError, KeyError, TypeError, AttributeError) as e:
#             print(f"Error fetching data for {ticker}: {e}")
#             continue

#     return value_df  # Return outside the loop


# def fetch_values_of_stocks(tickers):
#     value_cols = [
#         "Ticker",
#         "Price",
#         "PE-Ratio",
#         "PB-Ratio",
#         "PS-Ratio",
#         "EV/EBITDA",
#         "EV/GP"
#     ]
#     value_df = pd.DataFrame(columns=value_cols)
#     for ticker in tickers:
#      stock = yf.Ticker(ticker)
#      price = stock.history(period="1d")['Close'].iloc[-1]

#      financials = stock.financials
#      balanceSheet = stock.balance_sheet
#      cashflow = stock.cashflow

#      pe_ratio = stock.info.get("forwardPE",np.nan)
#      pb_ratio = stock.info.get("priceToBook",np.nan)
#      ps_ratio = stock.info.get("priceToSalesTrailing12months",np.nan)
#      ev = stock.info.get("Enterprisevalue",np.nan)
#      ebitda = stock.info.get("ebitda",np.nan)
#      evEbitda = ev / ebitda if ev and ebitda else np.nan
#      grossProfit = stock.info.get("grossMargins",np.nan)*stock.info.get("totalRevenue",np.nan)
#      evGrossProfit = ev / grossProfit if ev and grossProfit else np.nan
     
#      value_df.loc[len(value_df)] = [
#                 ticker,
#                 price,
#                 pe_ratio,
#                 pb_ratio,
#                 ps_ratio,
#                 evEbitda,
#                 evGrossProfit
#             ]
#      return value_df

def fetch_values_of_stocks(tickers):
    value_cols = [
        "Ticker", "Price", "PE-Ratio", "PB-Ratio", "PS-Ratio", "EV/EBITDA", "EV/GP"
    ]
    data = []  # Use a list of dictionaries for efficiency

    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            price = stock.history(period="1d")['Close'].iloc[-1] if not stock.history(period="1d").empty else np.nan

            pe_ratio = stock.info.get("forwardPE") # No default np.nan here, we want None if not available
            pb_ratio = stock.info.get("priceToBook")
            ps_ratio = stock.info.get("priceToSalesTrailing12months") or stock.info.get("priceToSales") # Try first priceToSalesTrailing12months, if not available try priceToSales
            ev = stock.info.get("enterpriseValue")
            ebitda = stock.info.get("ebitda")

            evEbitda = ev / ebitda if ev is not None and ebitda is not None and ebitda != 0 else np.nan
            grossProfit = stock.info.get("grossMargins") * stock.info.get("totalRevenue") if stock.info.get("grossMargins") is not None and stock.info.get("totalRevenue") is not None else np.nan
            evGrossProfit = ev / grossProfit if ev is not None and grossProfit is not None and grossProfit != 0 else np.nan

            row_data = {  # Create a dictionary for the row
                "Ticker": ticker,
                "Price": price,
                "PE-Ratio": pe_ratio,
                "PB-Ratio": pb_ratio,
                "PS-Ratio": ps_ratio,
                "EV/EBITDA": evEbitda,
                "EV/GP": evGrossProfit
            }
            data.append(row_data) # Append the dictionary
        except (IndexError, KeyError, TypeError, AttributeError) as e:
            print(f"Error fetching data for {ticker}: {e}")
            continue

    df = pd.DataFrame(data)  # Create the DataFrame *once*
    return df
                                        

In [178]:
tickers_list = tickers['Ticker'].values.tolist()
# tickers_list
df = fetch_values_of_stocks(tickers_list)
df

,Ticker,Price,PE-Ratio,PB-Ratio,PS-Ratio,EV/EBITDA,EV/GP
0,RELIANCE.NS,1264.599976,19.991770,2.156563,None,12.744551,6.269704
1,TCS.NS,4073.149902,26.937649,16.285955,None,21.836613,14.577731
2,HDFCBANK.NS,1690.949951,15.084299,2.814666,None,NaN,NaN
3,INFY.NS,1851.349976,25.690445,725.735000,None,1840.727966,1436.162285
4,ICICIBANK.NS,1255.550049,17.508322,3.442164,None,NaN,NaN
5,HINDUNILVR.NS,2506.050049,51.218210,11.496328,None,37.890420,16.753117
6,SBIN.NS,766.000000,9.201483,1.476353,None,NaN,NaN
7,BAJFINANCE.NS,8000.100098,23.668920,6.446448,None,NaN,22.708047
8,BHARTIARTL.NS,1623.250000,34.606537,10.783780,None,16.282887,11.919087
9,ITC.NS,462.549988,24.505194,7.690198,None,20.463970,12.272364


In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Ticker     50 non-null     object 
 1   Price      50 non-null     float64
 2   PE-Ratio   50 non-null     float64
 3   PB-Ratio   50 non-null     float64
 4   PS-Ratio   0 non-null      object 
 5   EV/EBITDA  42 non-null     float64
 6   EV/GP      44 non-null     float64
dtypes: float64(5), object(2)
memory usage: 2.9+ KB


In [180]:
value_cols = [
        "PE-Ratio",
        "PB-Ratio",
        "PS-Ratio",
        "EV/EBITDA",
        "EV/GP"
    ]
for col in value_cols:
    df[col] = df[col].fillna(df[col].mean())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Ticker     50 non-null     object 
 1   Price      50 non-null     float64
 2   PE-Ratio   50 non-null     float64
 3   PB-Ratio   50 non-null     float64
 4   PS-Ratio   0 non-null      float64
 5   EV/EBITDA  50 non-null     float64
 6   EV/GP      50 non-null     float64
dtypes: float64(6), object(1)
memory usage: 2.9+ KB


C:\Users\vadar\AppData\Local\Temp\ipykernel_19312\1238780930.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].mean())


## Finding the percentile of the stocks

In [181]:
percentile_metrics = {
    "PE-Ratio" : "PE-Ratio_Percentile", 
    "PB-Ratio" : "PB-Ratio_Percentile",
    "PS-Ratio" : "PS-Ratio_Percentile",
    "EV/EBITDA" : "EV/EBITDA_Percentile",
    "EV/GP" : "EV/GP_Percentile"
}


for metric, percentile in percentile_metrics.items():
    df[percentile] = df[metric].apply(lambda x: stats.percentileofscore(df[metric], x) / 100)

df.head()

,Ticker,Price,PE-Ratio,PB-Ratio,PS-Ratio,EV/EBITDA,EV/GP,PE-Ratio_Percentile,PB-Ratio_Percentile,PS-Ratio_Percentile,EV/EBITDA_Percentile,EV/GP_Percentile
0,RELIANCE.NS,1264.599976,19.991770,2.156563,NaN,12.744551,6.269704,0.38,0.22,NaN,0.24,0.24
1,TCS.NS,4073.149902,26.937649,16.285955,NaN,21.836613,14.577731,0.66,0.90,NaN,0.48,0.56
2,HDFCBANK.NS,1690.949951,15.084299,2.814666,NaN,109.258732,67.070933,0.22,0.24,NaN,0.89,0.91
3,INFY.NS,1851.349976,25.690445,725.735000,NaN,1840.727966,1436.162285,0.58,1.00,NaN,1.00,1.00
4,ICICIBANK.NS,1255.550049,17.508322,3.442164,NaN,109.258732,67.070933,0.34,0.36,NaN,0.89,0.91


## Determinig the Mean values for each ticker

In [182]:
from statistics import mean 

df['Value Score'] = df[[value for value in percentile_metrics.values()]].mean(axis = 1)

df

,Ticker,Price,PE-Ratio,PB-Ratio,PS-Ratio,EV/EBITDA,EV/GP,PE-Ratio_Percentile,PB-Ratio_Percentile,PS-Ratio_Percentile,EV/EBITDA_Percentile,EV/GP_Percentile,Value Score
0,RELIANCE.NS,1264.599976,19.991770,2.156563,NaN,12.744551,6.269704,0.38,0.22,NaN,0.24,0.24,0.2700
1,TCS.NS,4073.149902,26.937649,16.285955,NaN,21.836613,14.577731,0.66,0.90,NaN,0.48,0.56,0.6500
2,HDFCBANK.NS,1690.949951,15.084299,2.814666,NaN,109.258732,67.070933,0.22,0.24,NaN,0.89,0.91,0.5650
3,INFY.NS,1851.349976,25.690445,725.735000,NaN,1840.727966,1436.162285,0.58,1.00,NaN,1.00,1.00,0.8950
4,ICICIBANK.NS,1255.550049,17.508322,3.442164,NaN,109.258732,67.070933,0.34,0.36,NaN,0.89,0.91,0.6250
5,HINDUNILVR.NS,2506.050049,51.218210,11.496328,NaN,37.890420,16.753117,0.86,0.78,NaN,0.66,0.66,0.7400
6,SBIN.NS,766.000000,9.201483,1.476353,NaN,109.258732,67.070933,0.12,0.10,NaN,0.89,0.91,0.5050
7,BAJFINANCE.NS,8000.100098,23.668920,6.446448,NaN,109.258732,22.708047,0.48,0.62,NaN,0.89,0.74,0.6825
8,BHARTIARTL.NS,1623.250000,34.606537,10.783780,NaN,16.282887,11.919087,0.76,0.74,NaN,0.36,0.50,0.5900
9,ITC.NS,462.549988,24.505194,7.690198,NaN,20.463970,12.272364,0.54,0.68,NaN,0.44,0.52,0.5450


In [183]:
df = df.sort_values(by="Value Score", ascending=False)
df

,Ticker,Price,PE-Ratio,PB-Ratio,PS-Ratio,EV/EBITDA,EV/GP,PE-Ratio_Percentile,PB-Ratio_Percentile,PS-Ratio_Percentile,EV/EBITDA_Percentile,EV/GP_Percentile,Value Score
3,INFY.NS,1851.349976,25.690445,725.735000,NaN,1840.727966,1436.162285,0.58,1.00,NaN,1.00,1.00,0.8950
26,TITAN.NS,3552.000000,64.306430,32.347664,NaN,67.024323,27.398595,0.98,0.94,NaN,0.78,0.78,0.8700
48,DMART.NS,4023.750000,70.354740,14.003690,NaN,57.303538,29.922794,1.00,0.86,NaN,0.76,0.84,0.8650
13,HCLTECH.NS,1694.000000,24.196321,560.370500,NaN,1744.796103,1018.844787,0.52,0.98,NaN,0.98,0.98,0.8650
47,PIDILITIND.NS,2947.250000,61.155200,17.830027,NaN,48.954204,20.548212,0.96,0.92,NaN,0.72,0.72,0.8300
27,DIVISLAB.NS,5618.549805,56.739285,10.880122,NaN,56.448129,29.275896,0.92,0.76,NaN,0.74,0.82,0.8100
25,ADANIGREEN.NS,996.500000,55.857624,16.051352,NaN,30.848998,28.885826,0.88,0.88,NaN,0.62,0.80,0.7950
37,BRITANNIA.NS,5201.350098,50.245150,39.032470,NaN,39.784264,16.998566,0.82,0.96,NaN,0.68,0.68,0.7850
19,HDFCLIFE.NS,625.500000,59.395336,9.173438,NaN,99.309372,15.338639,0.94,0.72,NaN,0.80,0.62,0.7700
38,HAVELLS.NS,1656.199951,56.112717,13.937792,NaN,48.913200,14.450437,0.90,0.84,NaN,0.70,0.54,0.7450


## Top 10 stocks for Value Investing

In [184]:
df.head(10)

,Ticker,Price,PE-Ratio,PB-Ratio,PS-Ratio,EV/EBITDA,EV/GP,PE-Ratio_Percentile,PB-Ratio_Percentile,PS-Ratio_Percentile,EV/EBITDA_Percentile,EV/GP_Percentile,Value Score
3,INFY.NS,1851.349976,25.690445,725.735000,NaN,1840.727966,1436.162285,0.58,1.00,NaN,1.00,1.00,0.895
26,TITAN.NS,3552.000000,64.306430,32.347664,NaN,67.024323,27.398595,0.98,0.94,NaN,0.78,0.78,0.870
48,DMART.NS,4023.750000,70.354740,14.003690,NaN,57.303538,29.922794,1.00,0.86,NaN,0.76,0.84,0.865
13,HCLTECH.NS,1694.000000,24.196321,560.370500,NaN,1744.796103,1018.844787,0.52,0.98,NaN,0.98,0.98,0.865
47,PIDILITIND.NS,2947.250000,61.155200,17.830027,NaN,48.954204,20.548212,0.96,0.92,NaN,0.72,0.72,0.830
27,DIVISLAB.NS,5618.549805,56.739285,10.880122,NaN,56.448129,29.275896,0.92,0.76,NaN,0.74,0.82,0.810
25,ADANIGREEN.NS,996.500000,55.857624,16.051352,NaN,30.848998,28.885826,0.88,0.88,NaN,0.62,0.80,0.795
37,BRITANNIA.NS,5201.350098,50.245150,39.032470,NaN,39.784264,16.998566,0.82,0.96,NaN,0.68,0.68,0.785
19,HDFCLIFE.NS,625.500000,59.395336,9.173438,NaN,99.309372,15.338639,0.94,0.72,NaN,0.80,0.62,0.770
38,HAVELLS.NS,1656.199951,56.112717,13.937792,NaN,48.913200,14.450437,0.90,0.84,NaN,0.70,0.54,0.745
